### DiCE basic
- DiCE requires two inputs: a training dataset and a pre-trained ML model

In [1]:
# Import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

### ① Load data

In [2]:
df = helpers.load_adult_income_dataset()

In [3]:
df.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [4]:
# Description of transformed features
adult_info = helpers.get_adult_data_info()
adult_info

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [5]:
# Since continuous and discrete features have different ways of perturbation, we need to specify the names of the continuous features. 
# DiCE also requires the name of the output variable that the ML model will predict.

In [6]:
# Step 1: dice_ml.Data
d = dice_ml.Data(dataframe=df, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### ② Load ML model
- DiCE supports sklearn, tensorflow and pytorch models.
- The variable backend below indicates the implementation type of DiCE we want to use. Four backends are supported: sklearn, TensorFlow 1.x with backend='TF1', Tensorflow 2.x with backend='TF2', and PyTorch with backend='PYT'.
- Below we show use a trained classification model using sklearn.

In [7]:
# Split data into train and test
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

target = df["income"]
df_X = df.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(df_X, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 

from sklearn.compose import ColumnTransformer

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

### ③ Generate counterfactual examples using DiCE
- We now initialize the DiCE explainer, which needs a dataset and a model. 
- DiCE provides local explanation for the model m and requires an query input whose outcome needs to be explained.
- The method parameter specifies the explanation method. DiCE supports three methods for sklearn models: random sampling, genetic algorithm search, and kd-tree based generation.

In [8]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")

# Using method=random for generating CFs (Counterfactuals)
exp = dice_ml.Dice(d, m, method="random")

In [9]:
# Generate and visualize counterfactuals
e1 = exp.generate_counterfactuals(x_train[0:1], 
                                  total_CFs=2, 
                                  desired_class="opposite"
                                 )
e1.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,38,Private,HS-grad,Married,Blue-Collar,White,Male,44,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,Other/Unknown,Prof-school,-,-,-,-,-,1
1,-,-,Prof-school,-,Sales,-,-,-,1


In [10]:
# Changing only age and education
e2 = exp.generate_counterfactuals(x_train[0:1], 
                                  total_CFs=5, 
                                  desired_class="opposite",
                                  features_to_vary=["age", "education"]
                                  )
e2.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,38,Private,HS-grad,Married,Blue-Collar,White,Male,44,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,31.0,-,Bachelors,-,-,-,-,-,1
1,81.0,-,Doctorate,-,-,-,-,-,1
2,45.0,-,Doctorate,-,-,-,-,-,1
3,88.0,-,Prof-school,-,-,-,-,-,1
4,21.0,-,Bachelors,-,-,-,-,-,1


In [11]:
# Restricting age to be between [20,30] and Education to be either {'Doctorate', 'Prof-school'}.
e3 = exp.generate_counterfactuals(x_train[0:1], 
                                  total_CFs=5, 
                                  desired_class="opposite",
                                  permitted_range={'age':[20,30], 'education':['Doctorate', 'Prof-school']})
e3.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,38,Private,HS-grad,Married,Blue-Collar,White,Male,44,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Prof-school,-,-,-,-,57.0,1
1,-,-,Prof-school,-,-,-,-,25.0,1
2,-,Other/Unknown,Prof-school,-,-,-,-,-,1
3,-,-,Doctorate,-,-,-,-,92.0,1
4,-,-,Prof-school,-,-,-,-,74.0,1


### ④ Generate feature attributions (local and global) using DiCE

### Local feature importance scores
- These scores are computed for a given query instance (input point) by summarizing a set of counterfactual examples around the point.
- The total_CFs parameter denotes the number of counterfactuals that are used to create the local importance. More the better.

In [12]:
query_instance = x_train[0:1]
imp = exp.local_feature_importance(query_instance, total_CFs=10)

print(imp.local_importance)

[{'education': 1.0, 'occupation': 0.3, 'hours_per_week': 0.3, 'race': 0.2, 'workclass': 0.1, 'marital_status': 0.1, 'gender': 0.0, 'age': 0.0}]


### Global feature importance scores

- A global importance score per feature can be estimated by aggregating the scores over individual inputs. 
- The more the inputs, the better the estimate for global importance of a feature.

In [13]:
query_instances = x_train[0:20]
imp = exp.global_feature_importance(query_instances)

print(imp.summary_importance)

{'education': 0.56, 'marital_status': 0.41, 'hours_per_week': 0.325, 'occupation': 0.23, 'age': 0.23, 'workclass': 0.14, 'race': 0.105, 'gender': 0.08}
